<a href="https://colab.research.google.com/github/GreigFodd/Network-Data-Generation-Vanilla-GAN/blob/main/kerasgan_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# link & mount to google drive to import data
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# import libs
import pandas as pd
import keras

**Load in data**

In [ ]:
# import pre-processed data from google drive and store in dataframe using pandas function
df = pd.read_csv('/content/drive/MyDrive/Colab/ProjectData/data.csv')

In [ ]:
# print columns
df.columns

Index(['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'label', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s'],
      dtype='object')

In [ ]:
# ensure correct data was loaded
df['label'].value_counts()

BENIGN           1170495
DoS Hulk           70302
SSH-Patator         2922
DoS slowloris       2332
Heartbleed            11
Infiltration           4
Name: label, dtype: int64

In [ ]:
# split the dataset into 6 different datasets, one for each class
# GAN model will generate samples for one class at a time, therefore dataframe needs to be split by class. Will be joined again after all samples have been generated. 

df_benign = df.loc[df['label']=='BENIGN']
df_dosHulk = df.loc[df['label']=='DoS Hulk']
df_sshPatator = df.loc[df['label']=='SSH-Patator']
df_dosSlowloris = df.loc[df['label']=='DoS slowloris']
df_heartbleed = df.loc[df['label']=='Heartbleed']
df_infiltration = df.loc[df['label']=='Infiltration']

**Drop label category from each dataframe**

In [ ]:
df_benign = df_benign.drop(['label'], axis=1)
df_dosHulk = df_dosHulk.drop(['label'], axis=1)
df_sshPatator = df_sshPatator.drop(['label'], axis=1)
df_dosSlowloris = df_dosSlowloris.drop(['label'], axis=1)
df_heartbleed = df_heartbleed.drop(['label'], axis=1)
df_infiltration = df_infiltration.drop(['label'], axis=1)

**Apply power transformer**

In [ ]:
# Power transformer || https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html
# makes data more 'Gaussian-like' 

from sklearn.preprocessing import PowerTransformer
pw= PowerTransformer(method='yeo-johnson', standardize=True, copy=True)

In [ ]:
pwt_benign = pw.fit_transform(df_benign[df_benign.columns])
pwt_dosHulk = pw.fit_transform(df_dosHulk[df_dosHulk.columns])
pwt_sshPatator = pw.fit_transform(df_sshPatator[df_sshPatator.columns])
pwt_dosSlowloris = pw.fit_transform(df_dosSlowloris[df_dosSlowloris.columns])
pwt_heartbleed = pw.fit_transform(df_heartbleed[df_heartbleed.columns])
pwt_infiltration = pw.fit_transform(df_infiltration[df_infiltration.columns])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [ ]:
df_pwt_benign = pd.DataFrame(pwt_benign)
df_pwt_dosHulk = pd.DataFrame(pwt_dosHulk)
df_pwt_sshPatator = pd.DataFrame(pwt_sshPatator)
df_pwt_dosSlowloris = pd.DataFrame(pwt_dosSlowloris)
df_pwt_heartbleed = pd.DataFrame(pwt_heartbleed)
df_pwt_infiltration = pd.DataFrame(pwt_infiltration)

#df_pwt_benign.head()

In [ ]:
df_pwt_benign.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s']

df_pwt_dosHulk.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s'] 

df_pwt_sshPatator.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s']

df_pwt_dosSlowloris.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s']

df_pwt_heartbleed.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s']

df_pwt_infiltration.columns = ['init_win_bytes_fwd', 'subflow_fwd_bytes',
       'total_length_of_fwd_packets', 'fwd_packet_length_std',
       'bwd_packet_length_std', 'flow_iat_mean', 'fwd_iat_min', 'flow_iat_std',
       'flow_duration', 'active_mean', 'fwd_psh_flags', 'syn_flag_count',
       'fwd_packets_s']

In [ ]:
df_pwt_benign.head()

,init_win_bytes_fwd,subflow_fwd_bytes,total_length_of_fwd_packets,fwd_packet_length_std,bwd_packet_length_std,flow_iat_mean,fwd_iat_min,flow_iat_std,flow_duration,active_mean,fwd_psh_flags,syn_flag_count,fwd_packets_s
0,0.272716,-0.647020,-0.647020,-0.584707,-0.489364,-1.687959,-0.127557,-1.185899,-1.781634,-0.394253,-0.212966,-0.212966,1.778274
1,0.246321,-0.920325,-0.920325,-0.584707,-0.489364,-0.806958,-1.191152,-1.185899,-0.938415,-0.394253,-0.212966,-0.212966,0.865904
2,0.246321,-0.920325,-0.920325,-0.584707,-0.489364,-0.995693,-1.191152,-1.185899,-1.116171,-0.394253,-0.212966,-0.212966,1.032036
3,0.259953,-0.920325,-0.920325,-0.584707,-0.489364,-1.104276,-1.191152,-1.185899,-1.219155,-0.394253,-0.212966,-0.212966,1.125742
4,0.266436,-0.647020,-0.647020,-0.584707,-0.489364,-1.687959,-0.127557,-1.185899,-1.781634,-0.394253,-0.212966,-0.212966,1.778274


**Define model architecture**

In [ ]:
# model architecture adapted from https://github.com/ydataai/ydata-synthetic - a model for generating credit card data

# import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Model

from tensorflow.keras.optimizers import Adam

class GAN():
    
    def __init__(self, gan_args):
        [self.batch_size, lr, self.noise_dim,
         self.data_dim, layers_dim] = gan_args

        self.generator = Generator(self.batch_size).\
            build_model(input_shape=(self.noise_dim,), dim=layers_dim, data_dim=self.data_dim)

        self.discriminator = Discriminator(self.batch_size).\
            build_model(input_shape=(self.data_dim,), dim=layers_dim)

        optimizer = Adam(lr, 0.5)

        # Build and compile the discriminator
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # The generator takes noise as input and generates samples
        z = Input(shape=(self.noise_dim,))
        record = self.generator(z)

        # For the combined model only the generator is trained
        self.discriminator.trainable = False

        # The discriminator takes generated samples as input and determines validity
        validity = self.discriminator(record)

        # The combined model 
        # Trains the generator to try and create samples that will fool the discriminator 
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def get_data_batch(self, train, batch_size, seed=0):
        # # random sampling - some samples will have excessively low or high sampling, but easy to implement
        # np.random.seed(seed)
        # x = train.loc[ np.random.choice(train.index, batch_size) ].values
        # iterate through shuffled indices, so every sample gets covered evenly

        start_i = (batch_size * seed) % len(train)
        stop_i = start_i + batch_size
        shuffle_seed = (batch_size * seed) // len(train)
        np.random.seed(shuffle_seed)
        train_ix = np.random.choice(list(train.index), replace=False, size=len(train))  # wasteful to shuffle every time
        train_ix = list(train_ix) + list(train_ix)  # duplicate to cover ranges past the end of the set
        x = train.loc[train_ix[start_i: stop_i]].values
        return np.reshape(x, (batch_size, -1))
        
    def train(self, data, train_arguments):
        [cache_prefix, epochs, sample_interval] = train_arguments
        
        data_cols = data.columns

        # Adversarial ground truths
        valid = np.ones((self.batch_size, 1))
        fake = np.zeros((self.batch_size, 1))

        for epoch in range(epochs):    
            # Discriminator
            batch_data = self.get_data_batch(data, self.batch_size)
            noise = tf.random.normal((self.batch_size, self.noise_dim))

            # Generate a batch of new samples
            gen_data = self.generator.predict(noise)
    
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(batch_data, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_data, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
            # Generator
            noise = tf.random.normal((self.batch_size, self.noise_dim))
            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)
    
            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
    
    
class Generator():
    def __init__(self, batch_size):
        self.batch_size=batch_size
        
    def build_model(self, input_shape, dim, data_dim):
        input= Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim, activation='relu')(input)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dense(dim * 4, activation='relu')(x)
        x = Dense(data_dim)(x)
        return Model(inputs=input, outputs=x)

class Discriminator():
    def __init__(self,batch_size):
        self.batch_size=batch_size
    
    def build_model(self, input_shape, dim):
        input = Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim * 4, activation='relu')(input)
        x = Dropout(0.1)(x)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(dim, activation='relu')(x)
        x = Dense(1, activation='sigmoid')(x)
        return Model(inputs=input, outputs=x)

**Generating class data**

In [ ]:
#Define model params and training params

# set up dataframe for input into model

# EDIT lines 6 & 7 to choose which class of data to generate - change 'df_pwt_class'
data_columns = df_pwt_benign.columns
df_pwt_benign[data_columns] = df_pwt_benign[data_columns]

# set generator parametes
batch_size = 512
learning_rate=5e-4
noise_shape=32
input_shape=13
dim=128
 
# set training parameters 
epochs = 2500
log_step = 100

# assign to variable
generator_parameters = [batch_size, learning_rate, noise_shape, input_shape, dim]
training_parameters = ['', epochs, log_step]

In [ ]:
# training
synthesizer = GAN(generator_parameters)
# change passed in dataframe to select which class is to be generated
synthesizer.train(df_pwt_benign, training_parameters)

In [ ]:
models = {'GAN': ['GAN', False, synthesizer.generator]}

In [ ]:
np.random.seed(17) # used for retestability 

noise = np.random.normal(0,1, (1170495, 32)) # number of benign samples to be generated
#noise = np.random.normal(0,1, (70302, 32)) # number of dos hulk samples to be generated
#noise = np.random.normal(0,1, (2922, 32)) # number of ssh patator samples to be generated
#noise = np.random.normal(0,1, (2332, 32)) # number of dos slow loris samples to be generated
#noise = np.random.normal(0,1, (11, 32)) # number of heartbleed samples to be generated
#noise = np.random.normal(0,1, (4, 32)) # number of infiltration samples to be generated

[model_name, with_class, generator_model] = models['GAN']

X = generator_model.predict(noise)

In [ ]:
# Create a dataframe with generated samples, so can be used for evaluation later

gen_benign_samples = pd.DataFrame(X, columns=data_columns)
# gen_dosHulk_samples = pd.DataFrame(X, columns=data_columns)
# gen_sshPatator_samples = pd.DataFrame(X, columns=data_columns)
# gen_dosSlowloris_samples = pd.DataFrame(X, columns=data_columns)
# gen_heartbleed_samples = pd.DataFrame(X, columns=data_columns)
# gen_infiltration_samples = pd.DataFrame(X, columns=data_columns)

In [ ]:
gen_benign_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_benign_samples.csv')
# gen_dosHulk_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_dosHulk_samples.csv')
# gen_sshPatator_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_sshPatator_samples.csv')
# gen_dosSlowloris_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_dosSlowloris_samples.csv')
# gen_heartbleed_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_heartbleed_samples.csv')
# gen_infiltration_samples.to_csv(r'drive/My Drive/Colab/ProjectData/gen_infiltration_samples.csv')